In [ ]:
import torch
import torchvision

from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch import nn

In [ ]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        # Stage 1: Input -> Conv2d -> ReLU -> MaxPool2d -> Conv2d -> ReLU -> MaxPool2d
        self.Stage1 = nn.Sequential(
            nn.Conv2d(1, 6, 5, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=(2,2)),
            nn.Conv2d(6, 16, 3, padding='same'),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=(2,2))
        )
        # Stage 2: Stage_1 -> Flatten
        self.Stage2 = nn.Flatten()
        # Stage 3: Stage_2 -> Linear -> ReLU -> Linear -> Output
        self.Stage3 = nn.Sequential(
            nn.Linear(16*7*7, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )
    
    def forward(self, X):
        y_stage1 = self.Stage1(X)
        y_stage2 = self.Stage2(y_stage1)
        y_stage3 = self.Stage3(y_stage2)
        return y_stage3
    
    def train(self, training_data, batch_size=64, shuffle=True, optimizer="Adam", epochs=10, lr=1e-3):
        # Split dataset into batches for a training epoch
        training_dataloader = DataLoader(training_data, batch_size=batch_size, shuffle=shuffle)
        # Use Cross Entropy Loss
        loss_fn = nn.CrossEntropyLoss()
        # Use Adam or SGD for optimizing
        if optimizer.lower() == "adam":
            optimizer = torch.optim.Adam(self.parameters(), lr=lr)
        elif optimizer.lower() == "sgd":
            optimizer = torch.optim.SGD(self.parameters(), lr=lr)
        
        for i in range(epochs):
            for no, (X, y) in enumerate(training_dataloader):
                y_pre = self.forward(X)
                loss = loss_fn(y_pre, y)
                # Gradient descent
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

            print(f"Epoch {i}: Loss = {loss}")
    
    def Evaluate(self, testing_data):
        X, y = testing_data.data, testing_data.targets
        count, total = 0, 0

        for index in range(X.size()[0]):
            temp = torch.Tensor(X[index].numpy().reshape(1, 1, 28, 28))
            y_pre = self.forward(temp)
            total += 1
            if (y[index].item() == y_pre.argmax(1).item()):
                count += 1
        return count/total*100

In [ ]:
training_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=torchvision.transforms.ToTensor()
)
testing_data = torchvision.datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=torchvision.transforms.ToTensor()
)

In [ ]:
model = NeuralNetwork()
model.train(training_data, batch_size=128, epochs=20)

In [ ]:
model.Evaluate(testing_data)

In [ ]:
model.Evaluate(training_data)